In [1]:
import pickle
import os
import pandas as pd
import torch
from torch.utils.data import RandomSampler, DataLoader
import sklearn
import numpy as np
import random

from konlpy.tag import Mecab, Kkma, Okt, Komoran, Hannanum


In [2]:
import json
import os
import pickle
import time
from contextlib import contextmanager
from typing import List, NoReturn, Optional, Tuple, Union

import faiss
import numpy as np
import pandas as pd
from datasets import Dataset, concatenate_datasets, load_from_disk
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import (
    AutoConfig,
    AutoModelForQuestionAnswering,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    TrainingArguments,
    set_seed,
)

In [24]:

okt = Okt()
kma = Kkma()
komo = Komoran()
han = Hannanum() 
dataset = pd.read_csv('checkcheck.csv')


okt_noun = []
me_noun = []
kma_noun = []
komo_noun = []
han_noun = []
for i in dataset['question']:
    han_noun.append(" ".join(han.nouns(i)))
    kma_noun.append(" ".join(kma.nouns(i)))
    komo_noun.append(" ".join(komo.nouns(i)))



In [ ]:
#for i in dataset['question']:
#    print(i)

In [4]:
data_path = "../../data"
context_path = "wikipedia_documents.json"
with open(os.path.join(data_path, context_path), "r", encoding="utf-8") as f:
    wiki = json.load(f)

contexts = list(
    dict.fromkeys([v["text"] for v in wiki.values()])
)

In [8]:
han_contexts = []
for i in contexts:
    han_contexts.append(" ".join(han.nouns(i)))

java.lang.ArrayIndexOutOfBoundsException: Index 512 out of bounds for length 512
	at kr.ac.kaist.swrc.jhannanum.plugin.MajorPlugin.MorphAnalyzer.ChartMorphAnalyzer.MorphemeChart.analyzeUnknown(MorphemeChart.java:454)
	at kr.ac.kaist.swrc.jhannanum.plugin.MajorPlugin.MorphAnalyzer.ChartMorphAnalyzer.MorphemeChart.analyze(MorphemeChart.java:257)
	at kr.ac.kaist.swrc.jhannanum.plugin.MajorPlugin.MorphAnalyzer.ChartMorphAnalyzer.ChartMorphAnalyzer.processEojeol(ChartMorphAnalyzer.java:133)
	at kr.ac.kaist.swrc.jhannanum.plugin.MajorPlugin.MorphAnalyzer.ChartMorphAnalyzer.ChartMorphAnalyzer.morphAnalyze(ChartMorphAnalyzer.java:158)
	at kr.ac.kaist.swrc.jhannanum.hannanum.Workflow.analyzeInSingleThread(Workflow.java:825)
	at kr.ac.kaist.swrc.jhannanum.hannanum.Workflow.analyze(Workflow.java:521)
	at kr.lucypark.jhannanum.comm.HannanumInterface.simplePos09(Unknown Source)
java.lang.ArrayIndexOutOfBoundsException: Index 2046 out of bounds for length 2046
	at kr.ac.kaist.swrc.jhannanum.plugin.M

In [27]:
pickle_name = f"sparse_embedding100.bin"
tfidfv_name = f"tfidv100.bin"
emd_path = os.path.join(data_path, pickle_name)
tfidfv_path = os.path.join(data_path, tfidfv_name)

if os.path.isfile(emd_path) and os.path.isfile(tfidfv_path):
    with open(emd_path, "rb") as file:
        p_embedding100 = pickle.load(file)
    with open(tfidfv_path, "rb") as file:
        tfidfv100 = pickle.load(file)
    print("Embedding pickle load.")

Embedding pickle load.


In [21]:
#tokenizer = AutoTokenizer.from_pretrained(
#        "klue/bert-base",
#        use_fast=True,
#    )
def tokenizer(n):
    n = n.split(" ")
    return n

    

tfidf = TfidfVectorizer(
            tokenizer=tokenizer, ngram_range=(1, 2), max_features=1000000,
        )

In [22]:
context_vec = tfidf.fit_transform(han_contexts)
query_vec = tfidf.transform(han_noun)

In [29]:
a = pd.read_csv("./checkcheck.csv")
a= list(a['original_context'])
tmp = []
for v in a:
    for ii,vv in enumerate(contexts):
        if v == vv:
            tmp.append(ii)

In [23]:
    k = 1
    result = query_vec * context_vec.T

    for k in range(0, 100, 5):
        doc_scores = []
        doc_indices = []
        if not isinstance(result, np.ndarray):
            result = result.toarray()
        for i in range(result.shape[0]):
            sorted_result = np.argsort(result[i, :])[::-1]
            doc_scores.append(result[i, :][sorted_result].tolist()[:k])
            doc_indices.append(sorted_result.tolist()[:k])
        ccnt = 0
        indice = 0
        index = []
        for i, ii in zip(tmp, doc_indices):
            if i in ii:
             ccnt+=1
            else:
             index.append(indice)
            indice+=1
        print("top ",k," acc: ", ccnt/240*100)

top  0  acc:  0.0
top  5  acc:  13.750000000000002
top  10  acc:  15.833333333333332
top  15  acc:  18.333333333333332
top  20  acc:  20.416666666666668
top  25  acc:  21.666666666666668
top  30  acc:  21.666666666666668
top  35  acc:  22.5
top  40  acc:  22.916666666666664
top  45  acc:  22.916666666666664
top  50  acc:  22.916666666666664
top  55  acc:  22.916666666666664
top  60  acc:  22.916666666666664
top  65  acc:  22.916666666666664
top  70  acc:  23.333333333333332
top  75  acc:  23.333333333333332
top  80  acc:  23.333333333333332
top  85  acc:  23.75
top  90  acc:  24.166666666666668
top  95  acc:  24.166666666666668


In [16]:
tmp = [5294,
 47950,
 11891,
 55663,
 29185,
 23335,
 16171,
 54179,
 10001,
 16138,
 5983,
 5264,
 29979,
 9613,
 34822,
 52576,
 12748,
 53651,
 42566,
 25587,
 21562,
 4741,
 30932,
 42016,
 14233,
 9353,
 9855,
 44248,
 23856,
 34686,
 32363,
 12004,
 4792,
 49702,
 49502,
 47373,
 13194,
 4832,
 42712,
 51048,
 27704,
 40413,
 29834,
 30333,
 14801,
 15000,
 20988,
 5177,
 49540,
 33013,
 5287,
 15015,
 28253,
 23695,
 49489,
 34100,
 5068,
 41618,
 18935,
 5259,
 5269,
 4959,
 4804,
 5081,
 31254,
 29689,
 4963,
 42759,
 5705,
 4747,
 31555,
 20241,
 15653,
 8387,
 4663,
 29075,
 5224,
 7714,
 24486,
 46731,
 32778,
 16171,
 36572,
 27392,
 11676,
 55756,
 22337,
 55492,
 31963,
 31426,
 42467,
 21801,
 4651,
 53766,
 35358,
 48939,
 4583,
 25833,
 4846,
 5188,
 23291,
 42209,
 5256,
 23667,
 6925,
 23598,
 13676,
 23517,
 54652,
 21174,
 51565,
 4944,
 55273,
 16711,
 6258,
 29373,
 35175,
 8142,
 5159,
 8658,
 23623,
 55828,
 40862,
 21658,
 37140,
 4871,
 4834,
 25218,
 49488,
 5048,
 55702,
 4991,
 5067,
 3964,
 37476,
 5139,
 4948,
 25018,
 26055,
 55888,
 53318,
 53324,
 37389,
 31262,
 42568,
 5153,
 30722,
 12069,
 51001,
 48820,
 55500,
 54997,
 5238,
 18142,
 52190,
 4919,
 26381,
 20729,
 11558,
 24286,
 28583,
 4842,
 5103,
 25992,
 52707,
 7326,
 5263,
 37030,
 32220,
 4794,
 5298,
 52635,
 55840,
 7993,
 4938,
 37574,
 29001,
 20503,
 30775,
 54074,
 12031,
 5006,
 31148,
 41755,
 4620,
 33104,
 10976,
 49022,
 11017,
 5103,
 21571,
 14171,
 30877,
 5762,
 55699,
 16121,
 16383,
 4930,
 43859,
 13819,
 26999,
 12290,
 14631,
 11939,
 55243,
 5048,
 56430,
 38315,
 45113,
 39395,
 55399,
 6183,
 21751,
 22206,
 32530,
 23027,
 9735,
 8538,
 49874,
 29726,
 35401,
 40091,
 14415,
 44463,
 5617,
 4953,
 46708,
 37785,
 42569,
 23816,
 6315,
 4663,
 45254,
 51965,
 55851,
 49449,
 8387,
 4693,
 44764,
 4789]